In [1]:
import datarobot as dr
import datetime
import pandas as pd

In [ ]:
#import the training data set

filename = '/data/GDrive/Customers//Datasets/Regression/SALES_FORECASTING_BASE.csv'
now = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M')
project_name = 'ExampleRegression_{}'.format(now)
proj = dr.Project.create(sourcedata=filename,
                         project_name=project_name)
print('Project ID: {}'.format(proj.id))

In [ ]:
#set the target feature and run autopilot

proj.set_target(
    "TOTAL_REVENUE_NEXT_12M",
    mode=dr.enums.AUTOPILOT_MODE.QUICK,
    metric="RMSE",
    featurelist_id=proj.get_featurelists()[1].id,
    worker_count=-1
)

proj.wait_for_autopilot()

In [ ]:
#get the model that DR recommends for deployment

model = dr.ModelRecommendation.get(
    proj.id,
    dr.enums.RECOMMENDED_MODEL_TYPE.RECOMMENDED_FOR_DEPLOYMENT
).get_model()

model

In [ ]:
#Deploy the model to DR

#Get the prediction environment for native DR
prediction_server = dr.PredictionServer.list()[0]

#create the deployment

deployment = dr.Deployment.create_from_learning_model(
    model.id, label=ExampleRegression', description='Regression Model',
    default_prediction_server_id=prediction_server.id)

#enable drift tracking

deployment.update_drift_tracking_settings(feature_drift_enabled=True)


In [ ]:
#Score a file

dr.BatchPredictionJob.score_to_file(
    deployment.id,
    '/data/GDrive/Customers//Datasets/Regression/SALES_FORECASTING_V1_MIXED_NULL_Numerical_Categorical.csv',
    './predicted.csv',
)